In [40]:
import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework
import pandas as pd

In [41]:
# Login through local Wallaroo instance

wl = wallaroo.Client()


In [42]:
workspace_name="model-version-example"
pipeline_name="helper-demo-pipeline-1"
model_name="helper-demo-model-1"
model_file_name_01="./models/rf_model.onnx"
model_file_name_02="./models/gbr_model.onnx"

## Create Workspace

In [43]:
workspace = wl.get_workspace(workspace_name, create_if_not_exist=True)
wl.set_current_workspace(workspace)

{'name': 'model-version-example', 'id': 29, 'archived': False, 'created_by': 'fb2916bc-551e-4a76-88e8-0f7d7720a0f9', 'created_at': '2024-08-15T15:42:24.200005+00:00', 'models': [], 'pipelines': []}

In [44]:
workspace_id = workspace.id

## Upload Model with model_file_name_01



In [45]:
helper_model = (wl.upload_model(model_name, 
                                model_file_name_01, 
                                framework=Framework.ONNX)
                                .configure(tensor_fields=["tensor"])
                        )



In [46]:
display(helper_model)

Name,helper-demo-model-1
Version,a3f51eb7-ab86-46fa-80d2-e173ba3aae50
File Name,rf_model.onnx
SHA,e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-15-Aug 15:42:25
Workspace id,29
Workspace name,model-version-example


In [47]:
wl.search_models(model_name, workspace_name=workspace_name)

name,version,file_name,image_path,arch,accel,last_update_time,workspace_id,workspace_name
helper-demo-model-1,a3f51eb7-ab86-46fa-80d2-e173ba3aae50,rf_model.onnx,None,x86,none,2024-08-15 15:42:25.873240+00:00,29,model-version-example


## Build Pipeline and Deploy Model



In [48]:
pipeline = wl.build_pipeline("model-version-example")


pipeline.add_model_step(helper_model)
deploy_config = wallaroo.DeploymentConfigBuilder().cpus(0.25).memory("1Gi").build()
pipeline.deploy(deployment_config = deploy_config)

name,model-version-example
created,2024-08-15 15:42:27.847942+00:00
last_updated,2024-08-15 15:42:28.138090+00:00
deployed,True
workspace_id,29
workspace_name,model-version-example
arch,x86
accel,none
tags,
versions,"75f62f11-e292-42cd-840e-685b150f3fd5, c736de9b-fb0e-4add-8606-47d752d2e526"
steps,helper-demo-model-1


In [49]:
normal_input = pd.DataFrame.from_records({"tensor": [[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]]})
result = pipeline.infer(normal_input)
display(result)

,time,in.tensor,out.variable,anomaly.count
0,2024-08-15 15:42:42.488,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0,...",[718013.7],0


In [50]:
pipeline.undeploy()

name,model-version-example
created,2024-08-15 15:42:27.847942+00:00
last_updated,2024-08-15 15:42:28.138090+00:00
deployed,False
workspace_id,29
workspace_name,model-version-example
arch,x86
accel,none
tags,
versions,"75f62f11-e292-42cd-840e-685b150f3fd5, c736de9b-fb0e-4add-8606-47d752d2e526"
steps,helper-demo-model-1


## Upload Model as New Model Version



In [51]:
helper_model = (wl.upload_model(model_name, 
                                model_file_name_02, 
                                framework=Framework.ONNX)
                                .configure(tensor_fields=["tensor"])
                        )
display(helper_model)

Name,helper-demo-model-1
Version,57edf478-c1f5-495d-a543-ebf2b3c73a3f
File Name,gbr_model.onnx
SHA,ed6065a79d841f7e96307bb20d5ef22840f15da0b587efb51425c7ad60589d6a
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-15-Aug 15:43:22
Workspace id,29
Workspace name,model-version-example


## Display Model Versions

In [56]:
wl.search_models(workspace_name=workspace_name)

name,version,file_name,image_path,arch,accel,last_update_time,workspace_id,workspace_name
helper-demo-model-1,57edf478-c1f5-495d-a543-ebf2b3c73a3f,gbr_model.onnx,None,x86,none,2024-08-15 15:43:22.826953+00:00,29,model-version-example
helper-demo-model-1,a3f51eb7-ab86-46fa-80d2-e173ba3aae50,rf_model.onnx,None,x86,none,2024-08-15 15:42:25.873240+00:00,29,model-version-example


## Create New Pipeline Version with New Model Version

In [53]:
pipeline.clear()
pipeline.add_model_step(helper_model)
deploy_config = wallaroo.DeploymentConfigBuilder().cpus(0.25).memory("1Gi").build()
pipeline.deploy(deployment_config = deploy_config)

name,model-version-example
created,2024-08-15 15:42:27.847942+00:00
last_updated,2024-08-15 15:43:24.687114+00:00
deployed,True
workspace_id,29
workspace_name,model-version-example
arch,x86
accel,none
tags,
versions,"80c19355-80e6-49e3-8ca8-4c35cd50c314, 75f62f11-e292-42cd-840e-685b150f3fd5, c736de9b-fb0e-4add-8606-47d752d2e526"
steps,helper-demo-model-1


In [54]:
normal_input = pd.DataFrame.from_records({"tensor": [[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]]})
result = pipeline.infer(normal_input)
display(result)

,time,in.tensor,out.variable,anomaly.count
0,2024-08-15 15:43:39.821,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0,...",[704901.9],0


In [55]:
pipeline.undeploy()

name,model-version-example
created,2024-08-15 15:42:27.847942+00:00
last_updated,2024-08-15 15:43:24.687114+00:00
deployed,False
workspace_id,29
workspace_name,model-version-example
arch,x86
accel,none
tags,
versions,"80c19355-80e6-49e3-8ca8-4c35cd50c314, 75f62f11-e292-42cd-840e-685b150f3fd5, c736de9b-fb0e-4add-8606-47d752d2e526"
steps,helper-demo-model-1
